<a href="https://colab.research.google.com/github/dimdok9/prob_edu/blob/main/konlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 카카오톡 대화 토픽모델링

LDA, NMF를 이용하여 그룹대화방에 대해 토픽분석

In [4]:
# pip install konlpy

In [5]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import decomposition
from sklearn.decomposition import LatentDirichletAllocation
from konlpy.tag import Twitter

In [6]:

# Mount drive to access file

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# 대화 불러오기

In [7]:
import re

f = open('/content/drive/My Drive/Colab Notebooks/KakaoTalk_DSC.txt', 'r', encoding = 'UTF-8')
first = True

k = []
while True:
    
    if first == True:
        first = False
        f.readline()
    
    if not f.readline():
        break
            
    line = f.readline()
    
    if not re.match('---', line) and line != '\n':
        k.append(line.split(']'))
        try:
            k[-1][0] = k[-1][0].replace('[','')
            k[-1][1] = k[-1][1].replace('[','')
            k[-1][2] = k[-1][2].replace('\n','')
        except:
            pass

In [8]:
df = pd.DataFrame(k)
df.drop(3,axis = 1, inplace = True)
df.columns = ['작성자','시각','내용']
df.tail()

,작성자,시각,내용
8045,마사루,오후 5:53,책 제목은 신성모독수준이너
8046,DK,오전 9:48,https://m.bobaedream.co.kr/board/bbs_view/str...
8047,DK,오후 12:50,https://youtu.be/8WYHDfJDPDc
8048,DK,오후 9:26,사진
8049,Jinyoung,오전 11:03,유진 초이까...


In [9]:
okt_tag = Okt()

In [10]:
df.dropna(inplace = True)

In [11]:
f = open('/content/drive/My Drive/Colab Notebooks/kor_stopwords.txt', 'r', encoding = 'UTF-8')
lines = f.readlines()
kor_stopwords = []
for line in lines:
  kor_stopwords.append(line[:-1])
f.close()
kor_stopwords += ['.', '은', '개', 'ㅋㅋㅋㅋ', '2', '흠', 'ka', '한','...',  '걍','?', '근데', '서', '형', '하는','넘','머', '다', '하고','뭐', '내','는', '???', 'ㅡ','함', '만','안', '라거','도', '고','..','거', 'ㄹㅇ', '.....']
kor_stopwords += ['....', '아니거', '나도', '라고','인가','이여','ㅋㅋㅋ','거기','인','못','해서','한테','중','노','놈','??','데','그런거','햐','리']
kor_stopwords += ['님', '니', '글', '10', '면','타','하면','인데','햇','지','누','더','이나','이제','이라','그런','시', '말',]
kor_stopwords += [':', '!', '소','라', '-','임','있','게','!!!','하는데','이랑','기','랑']
kor_stopwords[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

In [24]:
from konlpy.tag import Twitter
t = Twitter()

def okt_tokenizer(text):
  # token = okt_tag.morphs(text)
  token = t.morphs(text)
  l = []
  for i in token:
    if i not in kor_stopwords:
      l.append(i)

  return l


okt_tokenizer('아 정말 재밌다')

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['정말', '재밌다']

In [28]:
tfidf_vectorizer = TfidfVectorizer(
                                   tokenizer = okt_tokenizer,
                                   ngram_range = (1,1), 
                                   min_df=2, 
                                   max_df=0.95,  
                                   max_features = 10000,
                                  )

tfidf_documents = tfidf_vectorizer.fit_transform(df['내용'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA 분석

In [29]:
n_topics = 15

lda = LatentDirichletAllocation(n_components = n_topics, random_state=0)
lda.fit(tfidf_documents)
topic_models = lda.components_

In [30]:
num_top_words = 10

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        term_list = [feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        print("topic %d:" % (topic_idx), term_list)


display_topics(lda, tfidf_feature_names, num_top_words)

topic 0: ['오이오이', '소리', '-_-;;', '가고', '싶', '진', '레알', 'ㅋ', '집', '살']
topic 1: ['씨벌', '후', '집', '아오', '.........', '대충', '이미', '솔직히', '했는데', '5252']
topic 2: ['52', '문제', '센', '가서', '조', '역시', '문', '대', '탁', '돈']
topic 3: ['좆', '로나', '그거', '쟞', '한국', '...........', '요새', '존나', '계사', '전']
topic 4: ['검색', '샵', '아직', '카이', '세', '스벌', '그것', 'ㅇㅅㅇ', '존나', '하기']
topic 5: ['까', '적', '맛', '같은', '상', '없', '난', '이다', '탈출', '수']
topic 6: ['씨발', 'ㅜ', '해', '새끼', '비', '음', '후', '어제', '와타시', '왔']
topic 7: ['사진', 'soda', '캬', '잘', '껄껄', 'ㅇㅋ', '하루', '케이', '죠', '예수님']
topic 8: ['오이', '했', '루', 'ㄱㄱ', '미국', '마사', '라멘', 'ㄴㄴ', '다시', '보다']
topic 9: ['같', '이모티콘', '짱', '엔', '전세', '…', '대너', '먹고', '연결', '조']
topic 10: ['지금', '오늘', '저녁', '똥', '너무', '진짜', '굿', '하노', '있다거', '가슴']
topic 11: ['시벌', '존나', 'da', '그냥', '처럼', '졸라', '와타시', '저런', '겠', '많이']
topic 12: ['그게', '키', '동영상', '댕기', '있으면', '이딴', '새', '좆', '사실', '하제']
topic 13: ['죠지', '오우', '죠', '........', 'socar', '바이', '씨', '닝겐', '상무', '뜨']
topic 14: ['와타시', '료헤

# NMF 분석

In [16]:
X = tfidf_documents
nmf = decomposition.NMF(n_components=n_topics, random_state=0, init="nndsvd")
# W = nmf.fit_transform(X) 
# H = nmf.components_

# top = 8
# topic_index_max = n_topics

# for topic_index in range(0, topic_index_max):
#     top_indices = np.argsort(H[topic_index, :])[::-1]
#     top_terms = []
#     for term_index in top_indices[0:top]:
#         top_terms.append(tfidf_feature_names[term_index])
#     print("topic ", topic_index, top_terms)